In [12]:
import requests
import json
import string
from scrapy.http import HtmlResponse





In [10]:
url = f"https://www.banxico.org.mx/ServiciosFinancieros/sanciones/publicaciones.xhtml?idDisposicion=3&idIntermediario=40&anio=2020&pecuniaria=true&tipoConsulta=I"

# payload = "companyname=Idea Engine"
# payload = {"nextValue": "2"}
# headers = {
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
#     # "authority": "www.mca.gov.in",
#     # "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
# }

res = requests.get(url)
resp = HtmlResponse("example.com",body=res.text,encoding='utf-8')


In [30]:
out_list = []
for y in range(2009,2022):
    url = f"https://www.banxico.org.mx/ServiciosFinancieros/sanciones/publicaciones.xhtml?idDisposicion=3&idIntermediario=40&anio={y}&pecuniaria=true&tipoConsulta=I"
    res = requests.get(url)
    resp = HtmlResponse("example.com",body=res.text,encoding='utf-8')
    for tr in resp.xpath("//table[@id='sanciones:sanciones']/tbody/tr"):
        name = tr.xpath("./td[1]/text()").get()
        sbody = tr.xpath("normalize-space(./td[2]/text())").get()
        # cmnt = tr.xpath("normalize-space(./td[3]/text())").get()
        tax_amount = tr.xpath("normalize-space(./td[4]/span/text())").get()

        print(tax_amount)

$18,091.20
$23,564.00
$41,655.20
$69,419.20
$69,419.20
$138,838.40
$50,964.00
$50,964.00
$48,908.80
$48,908.80
$22,614.00
$22,614.00
$69,419.20
$69,419.20
$22,929.60
$65,212.00
$88,141.60
$43,840.00
$43,840.00
$43,840.00
$43,840.00
$45,968.00
$19,766.40
$65,734.40
$43,840.00
$43,840.00
$54,800.00
$54,800.00
$85,912.00
$65,760.00
$25,052.80
$176,724.80
$19,766.40
$19,766.40
$43,840.00
$43,840.00
$43,840.00
$43,840.00
$43,840.00
$43,840.00
$46,580.00
$46,580.00
$43,840.00
$43,840.00
$42,072.00
$42,072.00
$85,912.00
$23,892.80
$109,804.80
$43,840.00
$43,840.00
$9,571.20
$9,571.20
$20,577.60
$191,424.00
$212,001.60
$183,872.00
$191,424.00
$375,296.00
$315,849.60
$315,849.60
$191,424.00
$191,424.00
$47,856.00
$191,424.00
$239,280.00
$183,872.00
$183,872.00
$19,765.60
$19,765.60
$30,339.20
$183,872.00
$214,211.20
$183,872.00
$183,872.00
$37,924.00
$37,924.00
$20,577.60
$20,577.60
$30,339.20
$303,388.80
$91,936.00
$31,584.80
$457,248.80
$9,571.20
$9,571.20
$9,571.20
$28,713.60
$36,848.80
$36,

In [29]:
for tr in resp.xpath("//table[@id='sanciones:sanciones']/tbody/tr"):
    name = tr.xpath("./td[1]/text()").get()
    sbody = tr.xpath("normalize-space(./td[2]/text())").get()
    # cmnt = tr.xpath("normalize-space(./td[3]/text())").get()
    tax_amount = tr.xpath("normalize-space(./td[4]/span/text())").get()
    print(tax_amount)

$13,102.40
$21,787.20
$65,462.48
$31,838.40
$51,481.60
$73,299.20
$256,971.28
$30,801.18
$319,212.80
$75,575.20
$425,589.18
$301,960.00
$241,568.00
$543,528.00
$233,728.00
$233,728.00
$28,146.40
$25,367.20
$25,778.40
$38,437.60
$41,908.80
$73,047.20
$62,147.20
$294,832.80
$40,758.40
$233,728.00
$233,728.00
$233,728.00
$741,942.40
$116,864.00
$11,992.00
$12,159.20
$12,367.20
$12,402.40
$60,828.80
$62,255.20
$107,664.00
$44,171.20
$51,867.20
$244,609.60
$737,180.80
$233,728.00
$241,568.00
$47,485.60
$74,710.40
$36,640.80
$35,260.00
$33,408.80
$34,804.00
$737,605.60
$13,746.40
$18,336.80
$32,083.20
$21,080.80
$21,080.80
$53,080.80
$103,616.00
$198,858.40
$14,216.00
$18,532.80
$27,149.60
$59,898.40


In [2]:
url = f"https://www.sebi.gov.in/sebiweb/ajax/home/getnewslistallinfo.jsp"

# payload = "companyname=Idea Engine"
payload = {"nextValue": "2"}
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
    # "authority": "www.mca.gov.in",
    # "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
}

res = requests.post(url, headers=headers, data=payload)

In [13]:
all_url = "https://suit.cibil.com/loadSearchResultPage?fileType=2&suitSearchBeanJson={%22borrowerName%22:%22%22,%22costAmount%22:%22%22,%22stateName%22:%22%22,%22directorName%22:%22%22,%22branchBean%22:null,%22dunsNumber%22:%22%22,%22city%22:%22%22,%22bankBean%22:{%22bankId%22:null,%22bankName%22:%22%22,%22categoryBean%22:{%22categoryId%22:0,%22categoryName%22:%22%22,%22categoryAllotedId%22:%22%22,%22active%22:0,%22enable%22:false},%22bankNoRecords%22:0,%22bankTotalAmount%22:%22%22,%22enable%22:false,%22active%22:0},%22quarterBean%22:{%22quarterId%22:null,%22quarterDate%22:null,%22quarterDateStr%22:%22%22,%22quarterName%22:%22%22,%22quarterMonthStr%22:%22%22,%22quarterYearStr%22:%22%22,%22isPush%22:0},%22stateBean%22:{%22stateId%22:null,%22stateName%22:%22%22,%22stateNoRecords%22:0,%22stateTotalAmount%22:%22%22,%22category%22:%22%22,%22enable%22:false,%22isActive%22:0},%22borrowerAddress%22:null,%22borrowerId%22:0,%22sort%22:0,%22totalRecords%22:0,%22totalAmount%22:%22%22,%22quarterCol%22:%22%22,%22categoryBean%22:null,%22noOFCGs1Cr%22:0,%22records1Cr%22:0,%22noOFCGs25Lac%22:0,%22records25Lac%22:0,%22cat%22:%22%22,%22catGroup%22:%22%22,%22fromQuarterId%22:0,%22toQuarterId%22:0,%22partyTypeId%22:0,%22quarterId%22:0,%22srNo%22:%22%22,%22userComments%22:%22%22,%22importDataBean%22:null,%22rejected%22:0,%22user%22:null,%22uploadBatchBean%22:null,%22rejectComment%22:%22%22,%22lastLimit%22:0,%22firstLimit%22:0,%22reject%22:null,%22edit%22:null,%22modify%22:null,%22editedTotalAmount%22:null,%22editedDirectorNames%22:null,%22rejectComments%22:null,%22updateReject%22:%22%22,%22userId%22:0,%22directorBean%22:null,%22isReview%22:%22%22,%22sortBy%22:null,%22sortOrder%22:null,%22summaryState%22:%221%22,%22summaryType%22:%221%22,%22directorId%22:0,%22directorSuffix%22:%22%22,%22dinNumber%22:%22%22,%22editedDirectorDin%22:null,%22dirPan%22:%22%22,%22editedDirectorPan%22:null,%22title%22:%22%22}&_search=false&nd=1653891673837&rows=10000&page="
p = "66"
k = "&sidx=&sord=asc"
res = requests.get(all_url+p+k)
data = json.loads(res.text)
print(data["records"])


927239


In [14]:
data["rows"][0]

{'borrowerName': 'Western India Industries Ltd',
 'costAmount': '',
 'stateName': 'NEW DELHI',
 'directorName': ' P.B.GADGIL--NA, N.P.GADGIL--NA, A.G.PANDIT--NA, B.T.GOKHALE--NA, D.P.JHA--NA, S.SEKHAR--NA',
 'branchBean': {'branchId': 1102,
  'branchName': 'NEW DELHI',
  'branchcode': '',
  'bankBean': {'bankId': 0,
   'bankName': 'STANDARD CHARTERED BANK',
   'categoryBean': {'categoryId': 0,
    'categoryName': '',
    'categoryAllotedId': '',
    'active': 0,
    'enable': False},
   'bankNoRecords': 0,
   'bankTotalAmount': '3050.00',
   'enable': False,
   'active': 0},
  'branchNoRecords': 0,
  'lastLimit': 0,
  'firstLimit': 0,
  'sortBy': None,
  'sortOrder': None,
  'active': 0,
  'enable': False},
 'dunsNumber': '',
 'city': '',
 'bankBean': {'bankId': 0,
  'bankName': 'STANDARD CHARTERED BANK',
  'categoryBean': {'categoryId': 0,
   'categoryName': '',
   'categoryAllotedId': '',
   'active': 0,
   'enable': False},
  'bankNoRecords': 0,
  'bankTotalAmount': '3050.00',
  'en

In [166]:
p = 1
maxp = 100
out_list = []
while True:
    print(f"-->{p}")
    if p >= maxp:
        break
    all_url = "https://suit.cibil.com/loadSearchResultPage?fileType=2&suitSearchBeanJson={%22borrowerName%22:%22%22,%22costAmount%22:%22%22,%22stateName%22:%22%22,%22directorName%22:%22%22,%22branchBean%22:null,%22dunsNumber%22:%22%22,%22city%22:%22%22,%22bankBean%22:{%22bankId%22:null,%22bankName%22:%22%22,%22categoryBean%22:{%22categoryId%22:0,%22categoryName%22:%22%22,%22categoryAllotedId%22:%22%22,%22active%22:0,%22enable%22:false},%22bankNoRecords%22:0,%22bankTotalAmount%22:%22%22,%22enable%22:false,%22active%22:0},%22quarterBean%22:{%22quarterId%22:null,%22quarterDate%22:null,%22quarterDateStr%22:%22%22,%22quarterName%22:%22%22,%22quarterMonthStr%22:%22%22,%22quarterYearStr%22:%22%22,%22isPush%22:0},%22stateBean%22:{%22stateId%22:null,%22stateName%22:%22%22,%22stateNoRecords%22:0,%22stateTotalAmount%22:%22%22,%22category%22:%22%22,%22enable%22:false,%22isActive%22:0},%22borrowerAddress%22:null,%22borrowerId%22:0,%22sort%22:0,%22totalRecords%22:0,%22totalAmount%22:%22%22,%22quarterCol%22:%22%22,%22categoryBean%22:null,%22noOFCGs1Cr%22:0,%22records1Cr%22:0,%22noOFCGs25Lac%22:0,%22records25Lac%22:0,%22cat%22:%22%22,%22catGroup%22:%22%22,%22fromQuarterId%22:0,%22toQuarterId%22:0,%22partyTypeId%22:0,%22quarterId%22:0,%22srNo%22:%22%22,%22userComments%22:%22%22,%22importDataBean%22:null,%22rejected%22:0,%22user%22:null,%22uploadBatchBean%22:null,%22rejectComment%22:%22%22,%22lastLimit%22:0,%22firstLimit%22:0,%22reject%22:null,%22edit%22:null,%22modify%22:null,%22editedTotalAmount%22:null,%22editedDirectorNames%22:null,%22rejectComments%22:null,%22updateReject%22:%22%22,%22userId%22:0,%22directorBean%22:null,%22isReview%22:%22%22,%22sortBy%22:null,%22sortOrder%22:null,%22summaryState%22:%221%22,%22summaryType%22:%221%22,%22directorId%22:0,%22directorSuffix%22:%22%22,%22dinNumber%22:%22%22,%22editedDirectorDin%22:null,%22dirPan%22:%22%22,%22editedDirectorPan%22:null,%22title%22:%22%22}&_search=false&nd=1653891673837&rows=10000&page="
    l = str(p)
    k = "&sidx=&sord=asc"
    res = requests.get(all_url+l+k)
    try:
        data = json.loads(res.text)
        maxp = int(data["total"])
        for j in data["rows"]:
            out_list.append(j["borrowerName"])
    except:
        try:
            res = requests.get(all_url+l+k)
            data = json.loads(res.text)
            maxp = int(data["total"])
            for j in data["rows"]:
                out_list.append(j["borrowerName"])
        except:
            print(f"failed for -{p}")
    p+=1
    # print(data["records"])

-->1
-->2
-->3
-->4
-->5
-->6
-->7
-->8
-->9
-->10
-->11
-->12
-->13
-->14
-->15
-->16
-->17
-->18
-->19
-->20
-->21
-->22
-->23
-->24
-->25
-->26
-->27
-->28
-->29
-->30
-->31
-->32
-->33
-->34
-->35
-->36
-->37
-->38
-->39
-->40
-->41
-->42
-->43
-->44
-->45
-->46
-->47
-->48
-->49
-->50
-->51
-->52
-->53
-->54
-->55
-->56
-->57
-->58
-->59
-->60
-->61
-->62
-->63
-->64
-->65
-->66


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [163]:
data["rows"][0]

{'borrowerName': 'INDIAN STEEL CO.',
 'costAmount': '',
 'stateName': 'MAHARASHTRA',
 'directorName': ' MILAPCHAND M. SANGHAVI--NA',
 'branchBean': {'branchId': 24767,
  'branchName': 'RECOVERY',
  'branchcode': '',
  'bankBean': {'bankId': 0,
   'bankName': 'ABHYUDAYA CO-OP. BANK LTD.',
   'categoryBean': {'categoryId': 0,
    'categoryName': '',
    'categoryAllotedId': '',
    'active': 0,
    'enable': False},
   'bankNoRecords': 0,
   'bankTotalAmount': '1044.25',
   'enable': False,
   'active': 0},
  'branchNoRecords': 0,
  'lastLimit': 0,
  'firstLimit': 0,
  'sortBy': None,
  'sortOrder': None,
  'active': 0,
  'enable': False},
 'dunsNumber': '',
 'city': '',
 'bankBean': {'bankId': 0,
  'bankName': 'ABHYUDAYA CO-OP. BANK LTD.',
  'categoryBean': {'categoryId': 0,
   'categoryName': '',
   'categoryAllotedId': '',
   'active': 0,
   'enable': False},
  'bankNoRecords': 0,
  'bankTotalAmount': '760.91',
  'enable': False,
  'active': 0},
 'quarterBean': {'quarterId': 0,
  'qua

In [145]:
data

{'total': 2147483647, 'records': 1, 'page': 1, 'rows': []}

In [135]:
import json
data = json.loads(res.text)["rows"]
len(data)

25022

In [136]:
d= json.loads(res.text)


In [139]:
import requests
import json

In [157]:
i = 1
out_list = []
total = 0
while(True):
    all_url = "https://suit.cibil.com/loadSearchResultPage?fileType=2&suitSearchBeanJson={%22borrowerName%22:%22%22,%22costAmount%22:%22%22,%22stateName%22:%22%22,%22directorName%22:%22%22,%22branchBean%22:null,%22dunsNumber%22:%22%22,%22city%22:%22%22,%22bankBean%22:{%22bankId%22:null,%22bankName%22:%22%22,%22categoryBean%22:{%22categoryId%22:0,%22categoryName%22:%22%22,%22categoryAllotedId%22:%22%22,%22active%22:0,%22enable%22:false},%22bankNoRecords%22:0,%22bankTotalAmount%22:%22%22,%22enable%22:false,%22active%22:0},%22quarterBean%22:{%22quarterId%22:null,%22quarterDate%22:null,%22quarterDateStr%22:%22%22,%22quarterName%22:%22%22,%22quarterMonthStr%22:%22%22,%22quarterYearStr%22:%22%22,%22isPush%22:0},%22stateBean%22:{%22stateId%22:"+str(i)+",%22stateName%22:%22%22,%22stateNoRecords%22:0,%22stateTotalAmount%22:%22%22,%22category%22:%22%22,%22enable%22:false,%22isActive%22:0},%22borrowerAddress%22:null,%22borrowerId%22:0,%22sort%22:0,%22totalRecords%22:0,%22totalAmount%22:%22%22,%22quarterCol%22:%22%22,%22categoryBean%22:null,%22noOFCGs1Cr%22:0,%22records1Cr%22:0,%22noOFCGs25Lac%22:0,%22records25Lac%22:0,%22cat%22:%22%22,%22catGroup%22:%22%22,%22fromQuarterId%22:0,%22toQuarterId%22:0,%22partyTypeId%22:0,%22quarterId%22:0,%22srNo%22:%22%22,%22userComments%22:%22%22,%22importDataBean%22:null,%22rejected%22:0,%22user%22:null,%22uploadBatchBean%22:null,%22rejectComment%22:%22%22,%22lastLimit%22:0,%22firstLimit%22:0,%22reject%22:null,%22edit%22:null,%22modify%22:null,%22editedTotalAmount%22:null,%22editedDirectorNames%22:null,%22rejectComments%22:null,%22updateReject%22:%22%22,%22userId%22:0,%22directorBean%22:null,%22isReview%22:%22%22,%22sortBy%22:null,%22sortOrder%22:null,%22summaryState%22:%221%22,%22summaryType%22:%221%22,%22directorId%22:0,%22directorSuffix%22:%22%22,%22dinNumber%22:%22%22,%22editedDirectorDin%22:null,%22dirPan%22:%22%22,%22editedDirectorPan%22:null,%22title%22:%22%22}&_search=false&nd=1653891673837&rows=5&page=1&sidx=&sord=asc"
    res = requests.get(all_url)
    print(print(f"--->{i}"))
    try:
        data = json.loads(res.text)
        print(f'{data["records"]}')
        print(f'{data["rows"][0]["stateName"]}')
        out_list.append(
            {
                "i" : i,
                "state" : data["rows"][0]["stateName"],
                "total" : data["records"]
            }
        )
        total+= int(data["records"])
    except:
        out_list.append(
            {
                "i" : i,
                "state" : "",
                "total" : ""
            }
        )
    i+=1
    if i==80:
        break
print(f"--------->>>{total}")

--->1
None
141
ANDAMAN & NICOBAR
--->2
None
54952
ANDHRA PRADESH
--->3
None
274
ARUNACHAL PRADESH
--->4
None
4383
ASSAM
--->5
None
147
BAHRAIN
--->6
None
7388
BIHAR
--->7
None
13234
CHANDIGARH
--->8
None
4
CHANDIGARH & DELHI
--->9
None
16
CHANDIGARH (U.T.) & DELHI
--->10
None
4660
CHHATTISGARH
--->11
None
102
DADRA & NAGAR HAVELI
--->12
None
131
DAMAN & DIU
--->13
None
89323
DELHI
--->14
None
2427
GOA
--->15
None
64913
GUJARAT
--->16
None
14446
HARYANA
--->17
None
2975
HIMACHAL PRADESH
--->18
None
204
HONG KONG
--->19
None
485
HYDERABAD
--->20
None
5215
JAMMU & KASHMIR
--->21
None
6990
JHARKHAND
--->22
None
41943
KARNATAKA
--->23
None
26314
KERALA
--->24
None
24
LAKSHADWEEP
--->25
None
120
LONDON
--->26
None
25832
MADHYA PRADESH
--->27
None
250681
MAHARASHTRA
--->28
None
58
MANIPUR
--->29
None
295
MEGHALAYA
--->30
None
87
MIZORAM
--->31
None
269
NAGALAND
--->32
None
14444
ORISSA
--->33
None
1022
PONDICHERRY
--->34
None
36529
PUNJAB
--->35
None
10618
RAJASTHAN
--->36
None
1
--->37
None


In [156]:
out_list

[{'i': 1, 'state': 'ANDAMAN & NICOBAR', 'total': 141},
 {'i': 2, 'state': 'ANDHRA PRADESH', 'total': 54943},
 {'i': 3, 'state': 'ARUNACHAL PRADESH', 'total': 274},
 {'i': 4, 'state': 'ASSAM', 'total': 4383},
 {'i': 5, 'state': 'BAHRAIN', 'total': 147},
 {'i': 6, 'state': 'BIHAR', 'total': 7388},
 {'i': 7, 'state': 'CHANDIGARH', 'total': 13234},
 {'i': 8, 'state': 'CHANDIGARH & DELHI', 'total': 4},
 {'i': 9, 'state': 'CHANDIGARH (U.T.) & DELHI', 'total': 16},
 {'i': 10, 'state': 'CHHATTISGARH', 'total': 4660},
 {'i': 11, 'state': 'DADRA & NAGAR HAVELI', 'total': 102},
 {'i': 12, 'state': 'DAMAN & DIU', 'total': 131},
 {'i': 13, 'state': 'DELHI', 'total': 89307},
 {'i': 14, 'state': 'GOA', 'total': 2427},
 {'i': 15, 'state': 'GUJARAT', 'total': 64904},
 {'i': 16, 'state': 'HARYANA', 'total': 14445},
 {'i': 17, 'state': 'HIMACHAL PRADESH', 'total': 2975},
 {'i': 18, 'state': 'HONG KONG', 'total': 204},
 {'i': 19, 'state': 'HYDERABAD', 'total': 485},
 {'i': 20, 'state': 'JAMMU & KASHMIR', 

In [85]:
len(data)

15

In [43]:
import string
from string import punctuation
l = list(string.ascii_lowercase) + [str(num) for num in list(range(10))]  + list(punctuation)
ml = [] 
for i in l:
    for k in l:
        for j in l:
            ml.append(i+k+j)

In [44]:
ml

314432

In [39]:
len(ml)

46656

In [34]:
ml

['aaa',
 'aab',
 'aac',
 'aad',
 'aae',
 'aaf',
 'aag',
 'aah',
 'aai',
 'aaj',
 'aak',
 'aal',
 'aam',
 'aan',
 'aao',
 'aap',
 'aaq',
 'aar',
 'aas',
 'aat',
 'aau',
 'aav',
 'aaw',
 'aax',
 'aay',
 'aaz',
 'aba',
 'abb',
 'abc',
 'abd',
 'abe',
 'abf',
 'abg',
 'abh',
 'abi',
 'abj',
 'abk',
 'abl',
 'abm',
 'abn',
 'abo',
 'abp',
 'abq',
 'abr',
 'abs',
 'abt',
 'abu',
 'abv',
 'abw',
 'abx',
 'aby',
 'abz',
 'aca',
 'acb',
 'acc',
 'acd',
 'ace',
 'acf',
 'acg',
 'ach',
 'aci',
 'acj',
 'ack',
 'acl',
 'acm',
 'acn',
 'aco',
 'acp',
 'acq',
 'acr',
 'acs',
 'act',
 'acu',
 'acv',
 'acw',
 'acx',
 'acy',
 'acz',
 'ada',
 'adb',
 'adc',
 'add',
 'ade',
 'adf',
 'adg',
 'adh',
 'adi',
 'adj',
 'adk',
 'adl',
 'adm',
 'adn',
 'ado',
 'adp',
 'adq',
 'adr',
 'ads',
 'adt',
 'adu',
 'adv',
 'adw',
 'adx',
 'ady',
 'adz',
 'aea',
 'aeb',
 'aec',
 'aed',
 'aee',
 'aef',
 'aeg',
 'aeh',
 'aei',
 'aej',
 'aek',
 'ael',
 'aem',
 'aen',
 'aeo',
 'aep',
 'aeq',
 'aer',
 'aes',
 'aet',
 'aeu',


In [33]:
len(ml)

17576

In [1]:
import json
data = json.load(open("MCA-ALL/mca-all-ds-2022-05-30.json"))

In [2]:
len(data)

2242293

In [8]:
import pandas as pd
pd.DataFrame(data).to_csv("mca-22.csv")

In [18]:
data[0]

{'companyID': 'U45200TG1997PTC027764',
 'companyName': 'AAA DEVELOPERS PRIVATE LIMITED'}

In [30]:
cids = []
cnames = []
for i in data:
    cids.append(i["companyID"])
    cnames.append(i["companyName"])

In [31]:
print(len(list(set(cids))))
print(len(list(set(cnames))))


799567
799153


In [21]:
import pandas as pd
raw_data = pd.DataFrame(data)

In [22]:
raw_data

,companyID,companyName
0,U45200TG1997PTC027764,AAA DEVELOPERS PRIVATE LIMITED
1,U65992HR1996PTC033148,AAA CHITS PVT. LTD.
2,U18101KL2005PTC018696,AAA APPARELS PRIVATE LIMITED
3,U00452MP1995PTC009503,AAA HOUSING FINANCE PVT LTD
4,U52599MP2002PTC015075,AAA NETWORK DISTRIBUTIONS PRIVATE LIMITED
...,...,...
830408,U70200HR2021PTC097871,999 TECHNOLOGY VENTURES PRIVATE LIMITED
830409,U51399TG2021PTC155515,999 PLYWOODBASKET.COM PRIVATE LIMITED
830410,U71290MH2022PTC376317,999 LEASING & INFRASTRUCTURE PRIVATE LIMITED
830411,AAF-8312,999 CREATIVE SERVICES LLP


In [23]:
r_d = raw_data.drop_duplicates()

In [24]:
r_d

,companyID,companyName
0,U45200TG1997PTC027764,AAA DEVELOPERS PRIVATE LIMITED
1,U65992HR1996PTC033148,AAA CHITS PVT. LTD.
2,U18101KL2005PTC018696,AAA APPARELS PRIVATE LIMITED
3,U00452MP1995PTC009503,AAA HOUSING FINANCE PVT LTD
4,U52599MP2002PTC015075,AAA NETWORK DISTRIBUTIONS PRIVATE LIMITED
...,...,...
830406,U92490MH2020PTC337765,999 RBC ENTERTAINMENT PRIVATE LIMITED
830407,U18109GJ2021PTC119722,999TEE (INDIA) PRIVATE LIMITED
830408,U70200HR2021PTC097871,999 TECHNOLOGY VENTURES PRIVATE LIMITED
830409,U51399TG2021PTC155515,999 PLYWOODBASKET.COM PRIVATE LIMITED


In [27]:
url = f"https://www.mca.gov.in/mcafoportal/companyLLPMasterData.do"
payload = {"companyName": "NAMELESS CREATIVES PRIVATE LIMITED",
"companyID": "U72900DL2017PTC327028",
"displayCaptcha": "true",
"userEnteredCaptcha": "snesors"}
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
    "authority": "www.mca.gov.in",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
}

res = requests.post(url, headers=headers, data=payload)



In [29]:
res.content

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /mcafoportal/companyLLPMasterData.do\non this server.</p>\n</body></html>\n'

In [28]:
data = json.loads(res.text)["companyList"]
data

[{'companyID': 'U45200TG1997PTC027764',
  'companyName': 'AAA DEVELOPERS PRIVATE LIMITED'},
 {'companyID': 'U65992HR1996PTC033148', 'companyName': 'AAA CHITS PVT. LTD.'},
 {'companyID': 'U18101KL2005PTC018696',
  'companyName': 'AAA APPARELS PRIVATE LIMITED'},
 {'companyID': 'U00452MP1995PTC009503',
  'companyName': 'AAA HOUSING FINANCE PVT LTD'},
 {'companyID': 'U52599MP2002PTC015075',
  'companyName': 'AAA NETWORK DISTRIBUTIONS PRIVATE LIMITED'},
 {'companyID': 'U65990MH1982PTC026796',
  'companyName': 'AAA INVESTMENT CO PVT LTD'},
 {'companyID': 'U28920MH1988PTC049710',
  'companyName': 'AAA ENGINEERING ENTERPRISES PRIVATE LIMITED'},
 {'companyID': 'U65910MH1991PTC064007',
  'companyName': 'AAA HOSPITALITY SERVICES PVT LTD'},
 {'companyID': 'U65923OR1994PLC003706',
  'companyName': 'AAA INVESTMENT AND FINANCE LTD'},
 {'companyID': 'U65921PB1990PTC010323', 'companyName': 'AAA FINANCE PVT LTD'},
 {'companyID': 'U70109PB2002PTC025120',
  'companyName': 'AAA INFRA VENTURES PRIVATE LIMIT

In [1]:
from deep_translator import GoogleTranslator
def langtranslation(to_translate):
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
    except:
        try:
            translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
        except:
            print(f">>>Translartion Bug : {to_translate}")
            translated = to_translate   
    return translated

# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [46]:
import camelot

In [3]:
tb = camelot.read_pdf("raw-files/file-157415407029476.pdf",pages="1",flavor="stream")
# tb = camelot.read_pdf("raw-files/file-157415407029476.pdf",pages="1",flavor="stream")


In [47]:
main = tb[0].df
new_header = main.iloc[0] #grab the first row for the header
main = main[1:] #take the data less the header row
main.columns = new_header #set the header row as the df header
main

,Arabic Name\n يبرعلاب مسلاا,English Name\n يزيلجنلااب مسلاا,Code\nزمرلا,,Arabic Address\n يبرعلاب ناونعلا,English Address\nيزيلجنلااب ناونعلا
1,يملاسلاا دلابلا,Al Bilad Islamic,BIQ034/2,,ةيرحلا ةحاس برق\n /\n ةداركلا\n /\n دادغب,Baghdad / Karrada / near Al Horriya square
2,يملاسلاا يميلقلاا نواعتلا,Islamic Regional Co-operation,BIQ036/2,,59\n ةيانب\n / \n17\n عراش\n / \n929\n ةلحم\n ...,Baghdad / Al Masbah / section 929 / street 17 ...
3,ناريا يلم,Melli Iran,BIQ037/2,,27\n قاقز\n / \n929\n ةلحم\n /\n تاصرعلا\n /\n...,Baghdad / Al Arasat / sectiom 929 / lane 27
4,نايسراب,Barsian,BIQ052/2,يملاسلاا ينطولا فرصملا لباقم\n /,69\n ةيانب\n /\n30\n عراش\n / \n929\n ةلحم\n /...,Baghdad / Karrada / Babil neighborhood / Al Ar...
5,تارفلاو هلجد,Dijlah wa Al Furat,EIQ0255/3,,19\n قاقز\n / \n110\n ةلحم\n /\n ديشرلا عراش\n...,Baghdad / Al Rasheed street / section 110 / la...
...,...,...,...,...,...,...
104,,Afaq Dubai,,,,Nishtiman Exchange Market
105,,Reydin,,,,Sheikhallah Exchange Market
106,,Al Khalidi Exchange,,,,Irbil / New Borsa / Nishtaman Building second ...
107,ءاطعلا,Al Attaa,BIQ034/2,,ةيرحلا ةحاس برق\n /\n ةداركلا\n /\n دادغب,Baghdad / Karrada / near Al Horriya square


In [48]:
import hashlib
from datetime import datetime
last_updated_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
def get_hash(n):
    return hashlib.sha256(((n+"Central Bank of Iraq Public Black List, Iraq" + "IRQ_T30064").lower()).encode()).hexdigest()


In [53]:
out_list1 = []
for i,row in main.astype(str).iterrows():
    pass_dict = {}
    name = row["English Name\n يزيلجنلااب مسلاا"]
    code = row["Code\nزمرلا"]
    pass_dict["uid"] = get_hash(code+name)
    pass_dict["name"] = name

    pass_dict["alias_name"] = [row["Arabic Name\n يبرعلاب مسلاا"]]

    pass_dict["list_type"] = "Entity"
    pass_dict["nns_status"] = "False"
    pass_dict["last_updated"] = last_updated_string
    pass_dict["list_id"] = "IRQ_T30064"
    pass_dict["country"] = ["Iraq"]

    comp = row["English Address\nيزيلجنلااب ناونعلا"]
    comp = comp.split(" / ")
    comp.reverse()
    comp = ",".join(comp)
    pass_dict["address"] = []
    pass_dict["address"].append({
        "complete_address": comp,
        "country" : "Iraq"
    })
    pass_dict["sanction_details"] = {}
    pass_dict["entity_details"] = {}
    pass_dict["comment"] = ""
    pass_dict["sanction_list"] = {}
    pass_dict["sanction_list"] = {
            "sl_authority": "Central Bank of Iraq, Iraq",
            "sl_url": "https://cbi.iq/news/view/741",
            "watch_list": "EMEA Watchlists",
            "sl_host_country": "Iraq",
            "sl_type": "Sanctions",
            "sl_source": "Central Bank of Iraq Public Black List, Iraq",
            "sl_description": "List of individuals and entities sanctioned by Central Bank of Iraq, Iraq"
        }

    out_list1.append(pass_dict)
    

In [50]:
out_list1[-1]
len(out_list1)


108

In [15]:
import pandas as pd
main = pd.read_excel("raw-files/Book 1.xlsx")

In [25]:
in_main = main[0:94]
en_main = main[95:]
en_main


,أسماء شركات التوسط,Name,Code,Arabic,English,Day,Month,Year
95,سلسلة الذهب,Selselat Al Thahab,TIQ026/1,كربﻻء,Karbala,15.0,5.0,2016.0
96,النبال العربية,Al Nibal Al Arabia,TIQ001/1,بغداد,Baghdad,8.0,1.0,2016.0
97,النهر الخالد,Al Nahr Al Khalid,TIQ004/1,بغداد,Baghdad,6.0,6.0,2017.0
98,الوائل,Al Wa'eel,TIQ019/1,بغداد,Baghdad,15.0,5.0,2016.0
99,مصرف العطاء اﻻسﻻمي (البﻻد سابقا),Al Ataa Islamic Bank,BIQ034/2,بغداد,Baghdad,16.0,5.0,2018.0
100,مصرف التعاون اﻷسﻻمي لﻷستثمار,Islamic Cooperation Bank for Investment,BIQ036/2,بغداد,Baghdad,8.0,1.0,2016.0
101,بنك ملي ايران,Bank Melli Iran,BIQ037/2,بغداد,Baghdad,8.0,1.0,2016.0
102,مصرف نور العراق اﻻسﻻمي لﻻستثمار \nوالتمويل,Iraq Noor Islamic Bank \nfor Investment & Fin...,BIQ064/2,بغداد,Baghdad,30.0,10.0,2019.0
103,بنك بارسيان,Parsian Bank,BIQ052/2,بغداد,Baghdad,8.0,1.0,2016.0
104,مصرف بابل,Babylon Bank,BIQ019/2,بغداد,Baghdad,24.0,12.0,2020.0


In [28]:
main.columns

Index(['أسماء شركات التوسط ', 'Name', 'Code', 'Arabic ', 'English', 'Day',
       'Month', 'Year'],
      dtype='object')

In [44]:
def tab_scrap(m,t):
    pass_list = []
    for i,r in m.astype(str).iterrows():
        name = r["Name"]
        code = r["Code"]
        uid = get_hash(name+code)

        alias = [r["أسماء شركات التوسط "]]

        comp = r["English"]
        adrs = [{
            "complete_address" : f"{comp.strip()},Iraq",
            "country" : "Iraq"
        }]

        date  = f"{r['Day']}/{r['Month']}/{r['Year']}"
        
        if t == "e":
            pass_list.append(
            {
                "uid":uid,
                "name":name,
                "alias_name": alias,
                "country": ["Iraq"],
                "list_type": "Entity",
                "last_updated": last_updated_string,
                "list_id": "IRQ_T30064",
                "entity_details": {},
                "nns_status": "False",
                "address": adrs,
                "sanction_details": {
                    "date" : date 
                                    },
                "comment":"",
                "sanction_list": {
                    "sl_authority": "Central Bank of Iraq, Iraq",
                    "sl_url": "https://cbi.iq/news/view/741",
                    "watch_list": "EMEA Watchlists",
                    "sl_host_country": "Iraq",
                    "sl_type": "Sanctions",
                    "sl_source": "Central Bank of Iraq Public Black List, Iraq",
                    "sl_description": "List of individuals and entities sanctioned by Central Bank of Iraq, Iraq"
                }
            }
            )
        else:   
            pass_list.append(
            {
                "uid":uid,
                "name":name,
                "alias_name": alias,
                "country": ["Iraq"],
                "list_type": "Individual",
                "last_updated": last_updated_string,
                "list_id": "IRQ_T30064",
                "individual_details": {},
                "nns_status": "False",
                "address": adrs,
                "sanction_details": {
                    "date" : date 
                                    },
                "comment":"",
                "sanction_list": {
                    "sl_authority": "Central Bank of Iraq, Iraq",
                    "sl_url": "https://cbi.iq/news/view/741",
                    "watch_list": "EMEA Watchlists",
                    "sl_host_country": "Iraq",
                    "sl_type": "Sanctions",
                    "sl_source": "Central Bank of Iraq Public Black List, Iraq",
                    "sl_description": "List of individuals and entities sanctioned by Central Bank of Iraq, Iraq"
                }
            }
            )
    return pass_list
        

In [45]:
out1 = tab_scrap(en_main,"e")
out2 = tab_scrap(in_main,"i")
print(len(out1))
print(len(out2))

10
94


In [54]:
main_out = out_list1 + out1 + out2
len(main_out)

212

In [58]:
import json
with open("sng-711.json", "w",encoding='utf-8') as outfile:
    json.dump(main_out, outfile, ensure_ascii=False, indent=4)